In [1]:
import os
import torch

# --- Main Paths ---
BASE_MODEL = "facebook/mbart-large-50"
DATA_PATH = "../../Dataset/new_large_CNN_dataset.csv" 
MODEL_OUTPUT_DIR = "mbart-large-50-cnn-summarizer-v15"
FINAL_SAVE_PATH = os.path.join(MODEL_OUTPUT_DIR, "final_model")

# --- Training Hyperparameters ---
LEARNING_RATE = 2e-5
NUM_EPOCHS = 4
BATCH_SIZE = 1
GRADIENT_ACCUMULATION_STEPS = 8
WEIGHT_DECAY = 0.1 # Regularization to prevent overfitting

# --- Evaluation & Generation ---
METRIC_FOR_BEST_MODEL = "eval_bleurt_f1"
MAX_INPUT_LENGTH = 1024
MAX_SUMMARY_LENGTH = 256
EVAL_BEAMS = 5

# --- Compute ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
import unicodedata
import pandas as pd
from datasets import Dataset, DatasetDict

def sanitize_text(text: str) -> str:
    """Basic text cleanup."""
    if not isinstance(text, str): 
        return ""
    return text.replace('""', '"').strip()

def normalize_text(text: str) -> str:
    """Normalize unicode characters."""
    if not isinstance(text, str): 
        return ""
    return ' '.join(unicodedata.normalize('NFKC', text).split())

def load_and_prep_dataset(data_path: str) -> DatasetDict:
    """Loads, cleans, formats, and splits the dataset."""
    
    # Load and clean
    df = pd.read_csv(data_path, engine='python', on_bad_lines='skip')
    df.dropna(subset=['raw_news_article', 'english_summary', 'hindi_summary'], inplace=True)
    
    for col in ['raw_news_article', 'english_summary', 'hindi_summary']:
        df[col] = df[col].apply(sanitize_text).apply(normalize_text)
    
    raw_dataset = Dataset.from_pandas(df)

    # Format for multilingual training
    processed_dataset = raw_dataset.map(
        _format_dataset_mbart, 
        batched=True, 
        remove_columns=raw_dataset.column_names
    )
    
    # Split
    train_test_split = processed_dataset.train_test_split(test_size=0.1, seed=42)
    return DatasetDict({
        'train': train_test_split['train'],
        'test': train_test_split['test']
    })

def _format_dataset_mbart(batch):
    """Duplicates each article for its English and Hindi summary."""
    inputs, targets, langs = [], [], []
    for article, eng_summary, hin_summary in zip(
        batch['raw_news_article'], batch['english_summary'], batch['hindi_summary']
    ):
        if isinstance(article, str) and article:
            inputs.append(article)
            targets.append(eng_summary)
            langs.append("en_XX")
            
            inputs.append(article)
            targets.append(hin_summary)
            langs.append("hi_IN")
            
    return {'article': inputs, 'summary': targets, 'target_lang': langs}

def tokenize_function(examples, tokenizer, max_input_len, max_summary_len):
    """Tokenizes articles (inputs) and summaries (labels)."""
    
    tokenizer.src_lang = "en_XX"
    model_inputs = tokenizer(
        examples['article'], 
        max_length=max_input_len, 
        truncation=True
    )
    
    labels_batch = []
    for i in range(len(examples['summary'])):
        tokenizer.tgt_lang = examples['target_lang'][i]
        labels = tokenizer(
            text_target=examples['summary'][i], 
            max_length=max_summary_len, 
            truncation=True
        )
        labels_batch.append(labels['input_ids'])
    
    model_inputs["labels"] = labels_batch
    return model_inputs

In [3]:
import evaluate
import numpy as np

# Initialize metrics globally to avoid reloading
rouge_metric = evaluate.load("rouge")
bleurt_metric = evaluate.load("bleurt", "bleurt-20")

def compute_metrics(eval_pred, tokenizer):
    """Decodes predictions and computes ROUGE and BLEURT scores."""
    predictions, labels = eval_pred
    
    predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Compute ROUGE
    rouge_result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels)
    
    # Compute BLEURT
    bleurt_result = bleurt_metric.compute(predictions=decoded_preds, references=decoded_labels)
    
    result = {
        "rouge1": rouge_result["rouge1"],
        "rouge2": rouge_result["rouge2"],
        "rougeL": rouge_result["rougeL"],
        "bleurt_f1": np.mean(bleurt_result["scores"])
    }
    
    return {k: round(v, 4) for k, v in result.items()}


INFO:tensorflow:Reading checkpoint C:\Users\admin\.cache\huggingface\metrics\bleurt\bleurt-20\downloads\extracted\8db8856a80394ae84b010e83ab663d4a3ccfa244ce3d0dbe00143f73e65ff123\BLEURT-20.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint BLEURT-20
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:BLEURT-20
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... max_seq_length:512
INFO:tensorflow:... vocab_file:None
INFO:tensorflow:... do_lower_case:None
INFO:tensorflow:... sp_model:sent_piece
INFO:tensorflow:... dynamic_seq_length:True
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Creating SentencePiece tokenizer.
INFO:tensorflow:Creating SentencePiece tokenizer.
INFO:tensorflow:Will load model: C:\Users\admin\.cache\huggingface\metrics\bleurt\bleurt-20\downloads\extracted\8db8856a80394ae84b010e83ab663d4a3ccfa244ce3d0dbe00143f73e65ff123\BLEURT-20\sent_piece.model.
INFO:tensorflow:Sentenc

INFO:tensorflow:BLEURT initialized.


In [6]:
import logging
import os
from functools import partial
from datetime import datetime
from transformers import (
    MBartForConditionalGeneration,
    MBart50TokenizerFast,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)

# Import custom modules
import config
import data_utils
import metrics_utils

def setup_logging():
    """Configures logging to file and stream."""
    os.makedirs(config.MODEL_OUTPUT_DIR, exist_ok=True)
    log_filename = os.path.join(
        config.MODEL_OUTPUT_DIR, 
        f"training_log_v15_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.log"
    )
    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s [%(levelname)s] - %(message)s",
        handlers=[
            logging.FileHandler(log_filename),
            logging.StreamHandler()
        ]
    )
    logging.info(f"--- Starting mBART v15 Training from Scratch ---")
    logging.info(f"Logging to: {log_filename}")

def main():
    setup_logging()

    # 1. Load and Prepare Data
    logging.info(f"Loading and processing data from: {config.DATA_PATH}")
    tokenized_datasets = data_utils.load_and_prep_dataset(config.DATA_PATH)
    logging.info(f"Dataset split: {len(tokenized_datasets['train'])} train, {len(tokenized_datasets['test'])} test")

    # 2. Load Base Model and Tokenizer
    logging.info(f"Loading base model and tokenizer from: {config.BASE_MODEL}")
    tokenizer = MBart50TokenizerFast.from_pretrained(config.BASE_MODEL)
    model = MBartForConditionalGeneration.from_pretrained(config.BASE_MODEL, use_safetensors=True)
    
    # 3. Tokenize Datasets
    logging.info("Tokenizing datasets...")
    tokenized_datasets = tokenized_datasets.map(
        partial(
            data_utils.tokenize_function,
            tokenizer=tokenizer,
            max_input_len=config.MAX_INPUT_LENGTH,
            max_summary_len=config.MAX_SUMMARY_LENGTH
        ),
        batched=True,
        remove_columns=['article', 'summary', 'target_lang']
    )
    
    # 4. Define Training Arguments
    logging.info("Configuring training arguments...")
    training_args = Seq2SeqTrainingArguments(
        output_dir=config.MODEL_OUTPUT_DIR,
        
        # Hyperparameters
        num_train_epochs=config.NUM_EPOCHS,
        learning_rate=config.LEARNING_RATE,
        per_device_train_batch_size=config.BATCH_SIZE,
        per_device_eval_batch_size=config.BATCH_SIZE,
        gradient_accumulation_steps=config.GRADIENT_ACCUMULATION_STEPS,
        weight_decay=config.WEIGHT_DECAY,
        
        # Logging and Saving (per epoch, as requested)
        logging_dir=os.path.join(config.MODEL_OUTPUT_DIR, "logs"),
        logging_strategy="epoch",
        eval_strategy="epoch", # Corrected from evaluation_strategy
        save_strategy="epoch",
        
        # We will run a separate script to find the best model
        load_best_model_at_end=False,
        save_total_limit=config.NUM_EPOCHS, # Save all checkpoints
        
        # Other settings
        predict_with_generate=True,
        fp16=config.DEVICE.type == 'cuda',
        report_to="tensorboard",
        generation_max_length=config.MAX_SUMMARY_LENGTH,
        generation_num_beams=config.EVAL_BEAMS,
    )
    
    # 5. Initialize Trainer
    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
    
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["test"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=partial(metrics_utils.compute_metrics, tokenizer=tokenizer),
    )

    # 6. Start Training
    logging.info(f"--- Training started for {config.NUM_EPOCHS} epochs ---")
    trainer.train()
    logging.info("--- Training finished successfully ---")
    logging.info(f"Checkpoints and logs are saved in: {config.MODEL_OUTPUT_DIR}")
    logging.info("Run 'evaluate.py' to find the best model and save it.")

if __name__ == "__main__":
    main()

Map:   0%|          | 0/9223 [00:00<?, ? examples/s]

Map:   0%|          | 0/16601 [00:00<?, ? examples/s]

Map:   0%|          | 0/1845 [00:00<?, ? examples/s]

C:\Users\admin\AppData\Local\Temp\ipykernel_27548\1931657635.py:96: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [7]:
import logging
import os
from functools import partial
from datetime import datetime
from transformers import (
    MBartForConditionalGeneration,
    MBart50TokenizerFast,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)

# Import custom modules
import config
import data_utils
import metrics_utils

def setup_logging():
    """Configures logging to file and stream."""
    os.makedirs(config.MODEL_OUTPUT_DIR, exist_ok=True)
    log_filename = os.path.join(
        config.MODEL_OUTPUT_DIR, 
        f"training_log_v15_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.log"
    )
    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s [%(levelname)s] - %(message)s",
        handlers=[
            logging.FileHandler(log_filename),
            logging.StreamHandler()
        ]
    )
    logging.info(f"--- Starting mBART v15 Training from Scratch ---")
    logging.info(f"Logging to: {log_filename}")

def main():
    setup_logging()

    # 1. Load and Prepare Data
    logging.info(f"Loading and processing data from: {config.DATA_PATH}")
    tokenized_datasets = data_utils.load_and_prep_dataset(config.DATA_PATH)
    logging.info(f"Dataset split: {len(tokenized_datasets['train'])} train, {len(tokenized_datasets['test'])} test")

    # 2. Load Base Model and Tokenizer
    logging.info(f"Loading base model and tokenizer from: {config.BASE_MODEL}")
    tokenizer = MBart50TokenizerFast.from_pretrained(config.BASE_MODEL)
    model = MBartForConditionalGeneration.from_pretrained(config.BASE_MODEL, use_safetensors=True)
    
    # 3. Tokenize Datasets
    logging.info("Tokenizing datasets...")
    tokenized_datasets = tokenized_datasets.map(
        partial(
            data_utils.tokenize_function,
            tokenizer=tokenizer,
            max_input_len=config.MAX_INPUT_LENGTH,
            max_summary_len=config.MAX_SUMMARY_LENGTH
        ),
        batched=True,
        remove_columns=['article', 'summary', 'target_lang']
    )
    
    # 4. Define Training Arguments
    logging.info("Configuring training arguments...")
    training_args = Seq2SeqTrainingArguments(
        output_dir=config.MODEL_OUTPUT_DIR,
        
        # Hyperparameters
        num_train_epochs=config.NUM_EPOCHS,
        learning_rate=config.LEARNING_RATE,
        per_device_train_batch_size=config.BATCH_SIZE,
        per_device_eval_batch_size=config.BATCH_SIZE,
        gradient_accumulation_steps=config.GRADIENT_ACCUMULATION_STEPS,
        weight_decay=config.WEIGHT_DECAY,
        
        # Logging and Saving (per epoch, as requested)
        logging_dir=os.path.join(config.MODEL_OUTPUT_DIR, "logs"),
        logging_strategy="epoch",
        eval_strategy="epoch", # Corrected from evaluation_strategy
        save_strategy="epoch",
        
        # We will run a separate script to find the best model
        load_best_model_at_end=False,
        save_total_limit=config.NUM_EPOCHS, # Save all checkpoints
        
        # Other settings
        predict_with_generate=True,
        fp16=config.DEVICE.type == 'cuda',
        report_to="tensorboard",
        generation_max_length=config.MAX_SUMMARY_LENGTH,
        generation_num_beams=config.EVAL_BEAMS,
    )
    
    # 5. Initialize Trainer
    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
    
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["test"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=partial(metrics_utils.compute_metrics, tokenizer=tokenizer),
    )

    # 6. Start Training
    logging.info(f"--- Training started for {config.NUM_EPOCHS} epochs ---")
    trainer.train()
    logging.info("--- Training finished successfully ---")
    logging.info(f"Checkpoints and logs are saved in: {config.MODEL_OUTPUT_DIR}")
    logging.info("Run 'evaluate.py' to find the best model and save it.")

if __name__ == "__main__":
    main()

Map:   0%|          | 0/9223 [00:00<?, ? examples/s]

Map:   0%|          | 0/16601 [00:00<?, ? examples/s]

Map:   0%|          | 0/1845 [00:00<?, ? examples/s]

C:\Users\admin\AppData\Local\Temp\ipykernel_27548\1931657635.py:96: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [9]:
import logging
import os
from functools import partial
from datetime import datetime
from transformers import (
    MBartForConditionalGeneration,
    MBart50TokenizerFast,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)

# Import custom modules
import config
import data_utils
import metrics_utils

def setup_logging():
    """Configures logging to file and stream."""
    os.makedirs(config.MODEL_OUTPUT_DIR, exist_ok=True)
    log_filename = os.path.join(
        config.MODEL_OUTPUT_DIR, 
        f"training_log_v15_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.log"
    )
    # Use force=True to re-configure logging in a notebook environment
    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s [%(levelname)s] - %(message)s",
        handlers=[
            logging.FileHandler(log_filename),
            logging.StreamHandler()
        ],
        force=True
    )
    logging.info(f"--- Starting mBART v15 Training ---")
    logging.info(f"Logging to: {log_filename}")

def main():
    setup_logging()

    # 1. Load and Prepare Data
    logging.info(f"Loading and processing data from: {config.DATA_PATH}")
    tokenized_datasets = data_utils.load_and_prep_dataset(config.DATA_PATH)
    logging.info(f"Dataset split: {len(tokenized_datasets['train'])} train, {len(tokenized_datasets['test'])} test")

    # 2. Load Base Model and Tokenizer
    logging.info(f"Loading base model and tokenizer from: {config.BASE_MODEL}")
    tokenizer = MBart50TokenizerFast.from_pretrained(config.BASE_MODEL)
    model = MBartForConditionalGeneration.from_pretrained(config.BASE_MODEL, use_safetensors=True)
    
    # 3. Tokenize Datasets
    logging.info("Tokenizing datasets...")
    tokenized_datasets = tokenized_datasets.map(
        partial(
            data_utils.tokenize_function,
            tokenizer=tokenizer,
            max_input_len=config.MAX_INPUT_LENGTH,
            max_summary_len=config.MAX_SUMMARY_LENGTH
        ),
        batched=True,
        remove_columns=['article', 'summary', 'target_lang']
    )
    
    # 4. Define Training Arguments
    logging.info("Configuring training arguments...")
    training_args = Seq2SeqTrainingArguments(
        output_dir=config.MODEL_OUTPUT_DIR,
        
        # Hyperparameters
        num_train_epochs=config.NUM_EPOCHS,
        learning_rate=config.LEARNING_RATE,
        per_device_train_batch_size=config.BATCH_SIZE,
        per_device_eval_batch_size=config.BATCH_SIZE,
        gradient_accumulation_steps=config.GRADIENT_ACCUMULATION_STEPS,
        weight_decay=config.WEIGHT_DECAY,
        
        # --- LOGGING FIX ---
        logging_dir=os.path.join(config.MODEL_OUTPUT_DIR, "logs"),
        logging_strategy="steps",   # Log every N steps
        logging_steps=150,          # Log training loss every 100 steps
        eval_strategy="epoch",      # Run full evaluation every epoch
        save_strategy="epoch",
        # ---------------------
        
        load_best_model_at_end=False,
        save_total_limit=config.NUM_EPOCHS, # Save all checkpoints
        
        # Other settings
        predict_with_generate=True,
        fp16=config.DEVICE.type == 'cuda',
        report_to="tensorboard",
        generation_max_length=config.MAX_SUMMARY_LENGTH,
        generation_num_beams=config.EVAL_BEAMS,
    )
    
    # 5. Initialize Trainer
    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
    
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["test"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=partial(metrics_utils.compute_metrics, tokenizer=tokenizer),
    )

    # 6. Start Training
    logging.info(f"--- Training started for {config.NUM_EPOCHS} epochs ---")
    trainer.train()
    logging.info("--- Training finished successfully ---")
    logging.info(f"Checkpoints and logs are saved in: {config.MODEL_OUTPUT_DIR}")
    logging.info("Run 'evaluate.py' to find the best model and save it.")

if __name__ == "__main__":
    main()
    

2025-11-13 18:21:14,270 [INFO] - --- Starting mBART v15 Training ---
2025-11-13 18:21:14,275 [INFO] - Logging to: mbart-large-50-cnn-summarizer-v15\training_log_v15_2025-11-13_18-21-14.log
2025-11-13 18:21:14,276 [INFO] - Loading and processing data from: ../../Dataset/new_large_CNN_dataset.csv


Map:   0%|          | 0/9223 [00:00<?, ? examples/s]

2025-11-13 18:21:17,640 [INFO] - Dataset split: 16601 train, 1845 test
2025-11-13 18:21:17,642 [INFO] - Loading base model and tokenizer from: facebook/mbart-large-50
2025-11-13 18:21:26,440 [INFO] - Tokenizing datasets...


Map:   0%|          | 0/16601 [00:00<?, ? examples/s]

Map:   0%|          | 0/1845 [00:00<?, ? examples/s]

2025-11-13 18:22:12,913 [INFO] - Configuring training arguments...
C:\Users\admin\AppData\Local\Temp\ipykernel_27548\1042724579.py:99: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
2025-11-13 18:22:14,899 [INFO] - --- Training started for 4 epochs ---


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
import logging
import os
from functools import partial
from datetime import datetime
from transformers import (
    MBartForConditionalGeneration,
    MBart50TokenizerFast,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)

# Import custom modules
import config
import data_utils
import metrics_utils

def setup_logging():
    """Configures logging to file and stream."""
    os.makedirs(config.MODEL_OUTPUT_DIR, exist_ok=True)
    log_filename = os.path.join(
        config.MODEL_OUTPUT_DIR, 
        f"training_log_v15_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.log"
    )
    # Use force=True to re-configure logging in a notebook environment
    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s [%(levelname)s] - %(message)s",
        handlers=[
            logging.FileHandler(log_filename)
            # REMOVED the StreamHandler(). This will stop the conflict
            # and allow the Trainer to log to the console by default.
        ],
        force=True
    )
    logging.info(f"--- Starting mBART v15 Training ---")
    logging.info(f"Logging to: {log_filename}")

def main():
    setup_logging()

    # 1. Load and Prepare Data
    logging.info(f"Loading and processing data from: {config.DATA_PATH}")
    final_datasets = data_utils.load_and_prep_dataset(config.DATA_PATH)
    logging.info(f"Dataset split: {len(final_datasets['train'])} train, {len(final_datasets['test'])} test")

    # 2. Load Base Model and Tokenizer
    logging.info(f"Loading base model and tokenizer from: {config.BASE_MODEL}")
    tokenizer = MBart50TokenizerFast.from_pretrained(config.BASE_MODEL)
    # Use safetensors=True to avoid the torch.load vulnerability error
    model = MBartForConditionalGeneration.from_pretrained(config.BASE_MODEL, use_safetensors=True)
    
    # 3. Tokenize Datasets
    logging.info("Tokenizing datasets...")
    # Use functools.partial to pass static arguments to the map function
    _tokenize_fn = partial(
        data_utils.tokenize_function,
        tokenizer=tokenizer,
        max_input_len=config.MAX_INPUT_LENGTH,
        max_summary_len=config.MAX_SUMMARY_LENGTH
    )
    tokenized_datasets = final_datasets.map(
        _tokenize_fn,
        batched=True,
        remove_columns=['article', 'summary', 'target_lang']
    )
    
    # 4. Define Training Arguments
    logging.info("Configuring training arguments...")
    training_args = Seq2SeqTrainingArguments(
        output_dir=config.MODEL_OUTPUT_DIR,
        
        # Hyperparameters
        num_train_epochs=config.NUM_EPOCHS,
        learning_rate=config.LEARNING_RATE,
        per_device_train_batch_size=config.BATCH_SIZE,
        per_device_eval_batch_size=config.BATCH_SIZE,
        gradient_accumulation_steps=config.GRADIENT_ACCUMULATION_STEPS,
        weight_decay=config.WEIGHT_DECAY,
        
        # --- LOGGING FIX ---
        logging_dir=os.path.join(config.MODEL_OUTPUT_DIR, "logs"),
        logging_strategy="steps",   # Log every N steps
        logging_steps=50,           # Log training loss every 50 steps
        log_level="info",           # This will now correctly print to the notebook
        eval_strategy="epoch",      # Run full evaluation every epoch
        save_strategy="epoch",
        # ---------------------
        
        load_best_model_at_end=False, # We run a separate script to evaluate
        save_total_limit=config.NUM_EPOCHS, # Save all checkpoints
        
        # Other settings
        predict_with_generate=True,
        fp16=config.DEVICE.type == 'cuda',
        report_to="tensorboard",
        generation_max_length=config.MAX_SUMMARY_LENGTH,
        generation_num_beams=config.EVAL_BEAMS,
    )
    
    # 5. Initialize Trainer
    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
    # Use partial to pass the tokenizer to the metrics function
    _compute_metrics_fn = partial(metrics_utils.compute_metrics, tokenizer=tokenizer)
    
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["test"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=_compute_metrics_fn,
    )

    # 6. Start Training
    logging.info(f"--- Training started for {config.NUM_EPOCHS} epochs ---")
    trainer.train()
    logging.info("--- Training finished successfully ---")
    logging.info(f"Checkpoints and logs are saved in: {config.MODEL_OUTPUT_DIR}")
    logging.info("Run 'evaluate.py' to find the best model and save it.")

if __name__ == "__main__":
    main()

Evaluation and Saving the best model

In [ ]:
import os
import json
import shutil
import logging
import config 
import logging_utils

def find_best_checkpoint():
    """
    Parses trainer_state.json files to find the best checkpoint
    and copies it to the final_model directory.
    """
    log_path = logging_utils.setup_logging(config.MODEL_OUTPUT_DIR, "evaluation_log_v15")
    print(f"--- Starting Evaluation & Save Script ---")
    print(f"Logging all output to: {log_path}\n")
    logging.info("--- Starting Post-Training Evaluation & Save Script ---")
    
    logging.info(f"Using '{config.METRIC_FOR_BEST_MODEL}' as the key metric.")
    print(f"Using '{config.METRIC_FOR_BEST_MODEL}' as the key metric.")
    
    best_metric_value = -1.0 
    best_checkpoint_path = None
    all_results = []

    checkpoint_dirs = [
        d for d in os.listdir(config.MODEL_OUTPUT_DIR) 
        if d.startswith("checkpoint-") and os.path.isdir(os.path.join(config.MODEL_OUTPUT_DIR, d))
    ]
    
    if not checkpoint_dirs:
        logging.error(f"No checkpoint directories found in {config.MODEL_OUTPUT_DIR}")
        print(f"Error: No checkpoint directories found in {config.MODEL_OUTPUT_DIR}")
        return

    logging.info(f"Found {len(checkpoint_dirs)} checkpoints to evaluate.")
    print(f"Found {len(checkpoint_dirs)} checkpoints to evaluate.")

    for chkpt_dir in checkpoint_dirs:
        state_path = os.path.join(config.MODEL_OUTPUT_DIR, chkpt_dir, "trainer_state.json")
        if not os.path.exists(state_path):
            logging.warning(f"No trainer_state.json found in {chkpt_dir}, skipping.")
            continue

        with open(state_path, "r") as f:
            state = json.load(f)
        
        eval_log = None
        for log in reversed(state["log_history"]):
            if config.METRIC_FOR_BEST_MODEL in log:
                eval_log = log
                break
        
        if eval_log:
            all_results.append(eval_log)
            metric_value = eval_log[config.METRIC_FOR_BEST_MODEL]
            
            if metric_value > best_metric_value:
                best_metric_value = metric_value
                best_checkpoint_path = os.path.join(config.MODEL_OUTPUT_DIR, chkpt_dir)
                logging.info(f"*** New best checkpoint: {chkpt_dir} ({config.METRIC_FOR_BEST_MODEL}: {metric_value}) ***")
                print(f"*** New best checkpoint: {chkpt_dir} ({config.METRIC_FOR_BEST_MODEL}: {metric_value}) ***")
        else:
            logging.warning(f"No evaluation metrics found in {state_path}.")

    # Log summary table
    summary_header = "\n" + "="*80 + "\n" + "--- FINAL EVALUATION SUMMARY ---".center(80) + "\n"
    table_header = f"{'Checkpoint':<20} | {'Step':<10} | {'Loss':<10} | {config.METRIC_FOR_BEST_MODEL:<12} | {'RougeL':<10}"
    summary_divider = "-" * len(table_header)
    
    logging.info(summary_header); print(summary_header)
    logging.info(table_header); print(table_header)
    logging.info(summary_divider); print(summary_divider)
    
    for log in sorted(all_results, key=lambda x: x['step']):
        name = f"checkpoint-{log['step']}"
        loss = log.get('eval_loss', 0.0)
        bleurt = log.get(config.METRIC_FOR_BEST_MODEL, 0.0)
        rougeL = log.get('eval_rougeL', 0.0)
        row = f"{name:<20} | {log['step']:<10} | {loss:<10.4f} | {bleurt:<12.4f} | {rougeL:<10.4f}"
        logging.info(row); print(row)
    
    logging.info("="*80 + "\n"); print("="*80 + "\n")

    # Save the best model
    if best_checkpoint_path:
        logging.info(f"Best model identified: {best_checkpoint_path}")
        print(f"Best model identified: {best_checkpoint_path}")
        
        if os.path.exists(config.FINAL_SAVE_PATH):
            logging.warning(f"Removing existing directory: {config.FINAL_SAVE_PATH}")
            print(f"Removing existing directory: {config.FINAL_SAVE_PATH}")
            shutil.rmtree(config.FINAL_SAVE_PATH)
            
        logging.info(f"Copying {best_checkpoint_path} to {config.FINAL_SAVE_PATH}...")
        print(f"Copying {best_checkpoint_path} to {config.FINAL_SAVE_PATH}...")
        shutil.copytree(best_checkpoint_path, config.FINAL_SAVE_PATH)
        
        logging.info("--- Best model saved successfully! ---")
        print("\n--- Best model saved successfully! ---")
        logging.info(f"You can now test it by running: python test.py")
        print(f"You can now test it by running: python test.py")
    else:
        logging.error("Could not determine the best model.")
        print("Error: Could not determine the best model.")

if __name__ == "__main__":
    try:
        find_best_checkpoint()
    except (KeyboardInterrupt):
        print("\n--- Evaluation Interrupted ---")
    except Exception as e:
        print(f"\n--- Evaluation FAILED --- \nError: {e}\nSee log file for details.")

Testing and Inferences

In [3]:
import torch
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import textwrap
import os
import config 
import logging_utils
import logging

model = None
tokenizer = None

def load_model():
    """Loads the final v15 model and tokenizer."""
    global model, tokenizer
    
    if not os.path.exists(config.FINAL_SAVE_PATH):
        print(f"Error: Model path not found: {config.FINAL_SAVE_PATH}")
        logging.error(f"Model path not found: {config.FINAL_SAVE_PATH}")
        print("Please run 'train.py' and 'evaluate.py' first.")
        return False

    print(f"Loading final model from: {config.FINAL_SAVE_PATH}...")
    logging.info(f"Loading final model from: {config.FINAL_SAVE_PATH}...")
    try:
        model = MBartForConditionalGeneration.from_pretrained(config.FINAL_SAVE_PATH).to(config.DEVICE)
        tokenizer = MBart50TokenizerFast.from_pretrained(config.FINAL_SAVE_PATH)
        model.eval()
        print(f"--- Model v15 loaded successfully on {config.DEVICE} ---")
        logging.info(f"--- Model v15 loaded successfully on {config.DEVICE} ---")
        return True
    except Exception as e:
        print(f"Error loading model: {e}")
        logging.error(f"Error loading model: {e}", exc_info=True)
        return False

def summarize_text(article_text: str):
    """Generates a high-quality summary."""
    if model is None or tokenizer is None:
        print("Model is not loaded. Please load the model first.")
        logging.error("summarize_text called before model was loaded.")
        return

    gen_kwargs = {
        "num_beams": 12,
        "length_penalty": 2.0,
        "repetition_penalty": 2.5,
        "no_repeat_ngram_size": 3,
        "do_sample": False,
        "early_stopping": True,
        "min_length": 30,
        "max_length": 250,
    }
    
    print("\n" + "="*80)
    print("SOURCE ARTICLE:")
    print(textwrap.fill(article_text, width=80))
    logging.info(f"--- Summarizing Source Article --- \n{article_text}\n")

    tokenizer.src_lang = "en_XX"
    inputs = tokenizer(article_text, return_tensors="pt", max_length=1024, truncation=True).to(config.DEVICE)

    # --- Generate English ---
    eng_summary_ids = model.generate(
        inputs.input_ids,
        forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"],
        **gen_kwargs
    )
    english_summary = tokenizer.decode(eng_summary_ids[0], skip_special_tokens=True)
    
    print("\n" + "="*80)
    print("GENERATED ENGLISH SUMMARY (v15):")
    print(textwrap.fill(english_summary, width=80))
    logging.info(f"--- Generated English Summary --- \n{english_summary}\n")

    # --- Generate Hindi ---
    hin_summary_ids = model.generate(
        inputs.input_ids,
        forced_bos_token_id=tokenizer.lang_code_to_id["hi_IN"],
        **gen_kwargs
    )
    hindi_summary = tokenizer.decode(hin_summary_ids[0], skip_special_tokens=True)
    
    print("\n" + "="*80)
    print("GENERATED HINDI SUMMARY (v15):")
    print(textwrap.fill(hindi_summary, width=80)) 
    logging.info(f"--- Generated Hindi Summary --- \n{hindi_summary}\n")
    print("\n" + "="*80)

def main():
    log_path = logging_utils.setup_logging(config.MODEL_OUTPUT_DIR, "testing_log_v15")
    print(f"--- Starting Test Script ---")
    print(f"Logging all output to: {log_path}\n")
    logging.info("--- Starting Test Script ---")
    
    if load_model():
        article_to_test = """
        A major tech firm today unveiled its latest flagship smartphone, 
        featuring a revolutionary new camera system with 'periscope zoom' 
        technology. The device, which also boasts a foldable OLED display 
        and 5G connectivity, aims to redefine the premium mobile market. 
        Analysts are optimistic, noting that the innovative camera could be 
        a key differentiator in a crowded field. However, concerns remain 
        about the device's high price point, which exceeds $1,500, 
        potentially limiting its mass-market appeal despite the advanced features.
        """
        summarize_text(article_to_test)

if __name__ == "__main__":
    try:
        main()
    except (KeyboardInterrupt):
        print("\n--- Test Interrupted ---")
    except Exception as e:
        print(f"\n--- Test FAILED --- \nError: {e}\nSee log file for details.")

loading configuration file mbart-large-50-cnn-summarizer-v15\final_model\config.json
Model config MBartConfig {
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "MBartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "dtype": "float32",
  "early_stopping": null,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL

--- Starting Test Script ---
Logging all output to: mbart-large-50-cnn-summarizer-v15\testing_log_v15_2025-11-14_07-35-31.log

Loading final model from: mbart-large-50-cnn-summarizer-v15\final_model...


loading weights file mbart-large-50-cnn-summarizer-v15\final_model\model.safetensors
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "pad_token_id": 1
}

loading configuration file mbart-large-50-cnn-summarizer-v15\final_model\generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": [
    2
  ],
  "forced_eos_token_id": 2,
  "max_length": 200,
  "num_beams": 5,
  "pad_token_id": 1
}

Could not locate the custom_generate/generate.py inside mbart-large-50-cnn-summarizer-v15\final_model.
loading file sentencepiece.bpe.model
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading file chat_template.jinja


--- Model v15 loaded successfully on cuda ---

SOURCE ARTICLE:
         A major tech firm today unveiled its latest flagship smartphone,
featuring a revolutionary new camera system with 'periscope zoom'
technology. The device, which also boasts a foldable OLED display          and
5G connectivity, aims to redefine the premium mobile market.          Analysts
are optimistic, noting that the innovative camera could be          a key
differentiator in a crowded field. However, concerns remain          about the
device's high price point, which exceeds $1,500,          potentially limiting
its mass-market appeal despite the advanced features.

GENERATED ENGLISH SUMMARY (v15):
A major tech firm has unveiled its latest flagship smartphone, featuring a
revolutionary new camera system with 'periscope zoom' technology. This device,
equipped with a foldable OLED display and 5G connectivity, aims to redefine the
premium mobile market. Analysts are optimistic, noting that the innovative
camera cou

In [4]:
import torch
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import textwrap
import os
import config 

model = None
tokenizer = None

def load_model():
    """Loads the final v15 model and tokenizer."""
    global model, tokenizer
    
    if not os.path.exists(config.FINAL_SAVE_PATH):
        print(f"Error: Model path not found: {config.FINAL_SAVE_PATH}")
        print("Please run 'train.py' and 'evaluate.py' first.")
        return False

    print(f"Loading final model from: {config.FINAL_SAVE_PATH}...")
    try:
        model = MBartForConditionalGeneration.from_pretrained(config.FINAL_SAVE_PATH).to(config.DEVICE)
        tokenizer = MBart50TokenizerFast.from_pretrained(config.FINAL_SAVE_PATH)
        model.eval()
        print(f"--- Model v15 loaded successfully on {config.DEVICE} ---")
        return True
    except Exception as e:
        print(f"Error loading model: {e}")
        return False

def summarize_text(article_text: str):
    """Generates a high-quality Hindi summary."""
    if model is None or tokenizer is None:
        print("Model is not loaded. Please load the model first.")
        return

    # Generation parameters are important for quality
    gen_kwargs = {
        "num_beams": 12,
        "length_penalty": 2.0,
        "repetition_penalty": 2.5,
        "no_repeat_ngram_size": 3,
        "do_sample": False,
        "early_stopping": True,
        "min_length": 30,
        "max_length": 250,
    }
    
    print("\n" + "="*80)
    print("SOURCE ARTICLE:")
    print(textwrap.fill(article_text, width=80))

    tokenizer.src_lang = "en_XX"
    inputs = tokenizer(article_text, return_tensors="pt", max_length=1024, truncation=True).to(config.DEVICE)

    # --- Generate Hindi Summary ---
    hin_summary_ids = model.generate(
        inputs.input_ids,
        forced_bos_token_id=tokenizer.lang_code_to_id["hi_IN"],
        **gen_kwargs
    )
    hindi_summary = tokenizer.decode(hin_summary_ids[0], skip_special_tokens=True)
    
    print("\n" + "="*80)
    print("GENERATED HINDI SUMMARY (v15):")
    print(textwrap.fill(hindi_summary, width=80)) 
    print("\n" + "="*80)

def main():
    print(f"--- Starting Test Script ---")
    
    if load_model():
        article_to_test = """
        A major tech firm today unveiled its latest flagship smartphone, 
        featuring a revolutionary new camera system with 'periscope zoom' 
        technology. The device, which also boasts a foldable OLED display 
        and 5G connectivity, aims to redefine the premium mobile market. 
        Analysts are optimistic, noting that the innovative camera could be 
        a key differentiator in a crowded field. However, concerns remain 
        about the device's high price point, which exceeds $1,500, 
        potentially limiting its mass-market appeal despite the advanced features.
        """
        summarize_text(article_to_test)

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\n--- Test Interrupted by user ---")
    except Exception as e:
        print(f"\n--- Test FAILED with an unexpected error: {e}")

loading configuration file mbart-large-50-cnn-summarizer-v15\final_model\config.json
Model config MBartConfig {
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "MBartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "dtype": "float32",
  "early_stopping": null,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL

--- Starting Test Script ---
Loading final model from: mbart-large-50-cnn-summarizer-v15\final_model...


loading configuration file mbart-large-50-cnn-summarizer-v15\final_model\generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": [
    2
  ],
  "forced_eos_token_id": 2,
  "max_length": 200,
  "num_beams": 5,
  "pad_token_id": 1
}

Could not locate the custom_generate/generate.py inside mbart-large-50-cnn-summarizer-v15\final_model.
loading file sentencepiece.bpe.model
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading file chat_template.jinja


--- Model v15 loaded successfully on cuda ---

SOURCE ARTICLE:
         A major tech firm today unveiled its latest flagship smartphone,
featuring a revolutionary new camera system with 'periscope zoom'
technology. The device, which also boasts a foldable OLED display          and
5G connectivity, aims to redefine the premium mobile market.          Analysts
are optimistic, noting that the innovative camera could be          a key
differentiator in a crowded field. However, concerns remain          about the
device's high price point, which exceeds $1,500,          potentially limiting
its mass-market appeal despite the advanced features.

GENERATED HINDI SUMMARY (v15):
एक प्रमुख टेक्स्ट फर्म ने अपनी नवीनतम फ्लैगशिप स्मार्टफोन का अनावरण किया है, जो
'पेरिस्कॉज़' तकनीक के साथ एक क्रांतिकारी नए कैमरे सिस्टम और foldable OLED
डिस्प्ले से चिह्नित है। यह स्मार्टफ़ोन प्रीमियम मोबाइल बाजार को फिर से परिभाषित
करने का लक्ष्य रखता है, और विश्लेषक आशावादी हैं कि यह एक भीड़भाड़ वाले क्षेत्र
में एक प

In [5]:
import torch
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import textwrap
import os
import config 

model = None
tokenizer = None

def load_model():
    """Loads the final v15 model and tokenizer."""
    global model, tokenizer
    
    if not os.path.exists(config.FINAL_SAVE_PATH):
        print(f"Error: Model path not found: {config.FINAL_SAVE_PATH}")
        print("Please run 'train.py' and 'evaluate.py' first.")
        return False

    # Minimal loading message
    print("Loading final model...")
    try:
        model = MBartForConditionalGeneration.from_pretrained(config.FINAL_SAVE_PATH).to(config.DEVICE)
        tokenizer = MBart50TokenizerFast.from_pretrained(config.FINAL_SAVE_PATH)
        model.eval()
        print("--- Model v15 Loaded ---")
        return True
    except Exception as e:
        print(f"Error loading model: {e}")
        return False

def summarize_text(article_text: str):
    """Generates a high-quality Hindi summary."""
    if model is None or tokenizer is None:
        print("Model is not loaded. Please load the model first.")
        return

    # Generation parameters are important for quality
    gen_kwargs = {
        "num_beams": 12,
        "length_penalty": 2.0,
        "repetition_penalty": 2.5,
        "no_repeat_ngram_size": 3,
        "do_sample": False,
        "early_stopping": True,
        "min_length": 30,
        "max_length": 250,
    }
    
    print("\n" + "="*80)
    print("SOURCE ARTICLE:")
    print(textwrap.fill(article_text, width=80))

    tokenizer.src_lang = "en_XX"
    inputs = tokenizer(article_text, return_tensors="pt", max_length=1024, truncation=True).to(config.DEVICE)

    # --- Generate Hindi Summary ---
    hin_summary_ids = model.generate(
        inputs.input_ids,
        forced_bos_token_id=tokenizer.lang_code_to_id["hi_IN"],
        **gen_kwargs
    )
    hindi_summary = tokenizer.decode(hin_summary_ids[0], skip_special_tokens=True)
    
    print("\n" + "="*80)
    print("GENERATED HINDI SUMMARY (v15):")
    print(textwrap.fill(hindi_summary, width=80)) 
    print("\n" + "="*80)

def main():
    if load_model():
        article_to_test = """
        A major tech firm today unveiled its latest flagship smartphone, 
        featuring a revolutionary new camera system with 'periscope zoom' 
        technology. The device, which also boasts a foldable OLED display 
        and 5G connectivity, aims to redefine the premium mobile market. 
        Analysts are optimistic, noting that the innovative camera could be 
        a key differentiator in a crowded field. However, concerns remain 
        about the device's high price point, which exceeds $1,500, 
        potentially limiting its mass-market appeal despite the advanced features.
        """
        summarize_text(article_to_test)

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\n--- Test Interrupted by user ---")
    except Exception as e:
        print(f"\n--- Test FAILED with an unexpected error: {e}")

loading configuration file mbart-large-50-cnn-summarizer-v15\final_model\config.json
Model config MBartConfig {
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "MBartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "dtype": "float32",
  "early_stopping": null,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL

Loading final model...


loading configuration file mbart-large-50-cnn-summarizer-v15\final_model\generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": [
    2
  ],
  "forced_eos_token_id": 2,
  "max_length": 200,
  "num_beams": 5,
  "pad_token_id": 1
}

Could not locate the custom_generate/generate.py inside mbart-large-50-cnn-summarizer-v15\final_model.
loading file sentencepiece.bpe.model
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading file chat_template.jinja


--- Model v15 Loaded ---

SOURCE ARTICLE:
         A major tech firm today unveiled its latest flagship smartphone,
featuring a revolutionary new camera system with 'periscope zoom'
technology. The device, which also boasts a foldable OLED display          and
5G connectivity, aims to redefine the premium mobile market.          Analysts
are optimistic, noting that the innovative camera could be          a key
differentiator in a crowded field. However, concerns remain          about the
device's high price point, which exceeds $1,500,          potentially limiting
its mass-market appeal despite the advanced features.

GENERATED HINDI SUMMARY (v15):
एक प्रमुख टेक्स्ट फर्म ने अपनी नवीनतम फ्लैगशिप स्मार्टफोन का अनावरण किया है, जो
'पेरिस्कॉज़' तकनीक के साथ एक क्रांतिकारी नए कैमरे सिस्टम और foldable OLED
डिस्प्ले से चिह्नित है। यह स्मार्टफ़ोन प्रीमियम मोबाइल बाजार को फिर से परिभाषित
करने का लक्ष्य रखता है, और विश्लेषक आशावादी हैं कि यह एक भीड़भाड़ वाले क्षेत्र
में एक प्रमुख विनिर्देशक बन स

In [9]:
import torch
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import textwrap
import os
import config

model = None
tokenizer = None
gen_kwargs = {}

def load_model_and_settings():
    """Loads the v15 model, tokenizer, and high-quality generation parameters."""
    global model, tokenizer, gen_kwargs
    
    if model is not None:
        print("Model is already loaded.")
        return True

    if not os.path.exists(config.FINAL_SAVE_PATH):
        print(f"Error: Model path not found: {config.FINAL_SAVE_PATH}")
        print("Please run 'train.py' and 'evaluate.py' first.")
        return False

    print("Loading final model...")
    try:
        model = MBartForConditionalGeneration.from_pretrained(config.FINAL_SAVE_PATH).to(config.DEVICE)
        tokenizer = MBart50TokenizerFast.from_pretrained(config.FINAL_SAVE_PATH)
        model.eval()
        
        gen_kwargs = {
            "num_beams": 12,
            "length_penalty": 2.0,
            "repetition_penalty": 2.5,
            "no_repeat_ngram_size": 3,
            "do_sample": False,
            "early_stopping": True,
            "min_length": 30,
            "max_length": 250,
        }
        
        print(f"--- Model v15 Loaded on {config.DEVICE} ---")
        return True
    except Exception as e:
        print(f"Error loading model: {e}")
        return False

def summarize_hindi(article_text: str):
    """Generates a high-quality Hindi summary for the given text."""
    if model is None or tokenizer is None:
        print("Model is not loaded. Please run the setup cell (Cell 1) first.")
        return

    print("\n" + "="*80)
    print("SOURCE ARTICLE:")
    print(textwrap.fill(article_text, width=80))

    tokenizer.src_lang = "en_XX"
    inputs = tokenizer(article_text, return_tensors="pt", max_length=1024, truncation=True).to(config.DEVICE)

    # Generate Hindi Summary
    hin_summary_ids = model.generate(
        inputs.input_ids,
        forced_bos_token_id=tokenizer.lang_code_to_id["hi_IN"],
        **gen_kwargs  
    )
    hindi_summary = tokenizer.decode(hin_summary_ids[0], skip_special_tokens=True)
    
    print("\n" + "="*80)
    print("GENERATED HINDI SUMMARY (v15):")
    print(textwrap.fill(hindi_summary, width=80)) 
    print("\n" + "="*80)

load_model_and_settings()

loading configuration file mbart-large-50-cnn-summarizer-v15\final_model\config.json
Model config MBartConfig {
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "MBartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "dtype": "float32",
  "early_stopping": null,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL

Loading final model...


loading configuration file mbart-large-50-cnn-summarizer-v15\final_model\generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": [
    2
  ],
  "forced_eos_token_id": 2,
  "max_length": 200,
  "num_beams": 5,
  "pad_token_id": 1
}

Could not locate the custom_generate/generate.py inside mbart-large-50-cnn-summarizer-v15\final_model.
loading file sentencepiece.bpe.model
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading file chat_template.jinja


--- Model v15 Loaded on cuda ---


True

In [12]:
articles_to_test = [
    # 1. Sports
    """India secured a decisive victory over Australia in the final match of the T20 series, winning by a margin of 35 runs in Bengaluru. Batting first, India posted a competitive total of 198 for 4, thanks to a powerful half-century from captain Suryakumar Yadav.""",
    
    # 2. Science
    """NASA's Artemis program achieved a major milestone this week as the Orion spacecraft successfully completed its uncrewed flyby of the Moon and is now on its return trajectory to Earth. The mission, Artemis I, is a critical test of the agency's deep space exploration systems.""",
    
    # 3. Environment
    """A new international report has found that the rate of deforestation in the Amazon rainforest accelerated by nearly 20% in the last year, reaching its highest level in over a decade. The report attributes the surge to increased illegal logging and agricultural expansion.""",
    
    # 4. Entertainment
    """The highly anticipated sequel to a blockbuster science fiction film has officially begun production, with the studio releasing the first on-set photo. The image features the return of the original cast members alongside several new additions.""",
    
    # 5. Automotive/Tech
    """A legacy automaker has announced a massive $50 billion investment into its electric vehicle (EV) division, signaling a dramatic acceleration of its transition away from internal combustion engines. The company plans to launch 15 new all-electric models over the next three years.""",
    
    # 6. World News
    """Diplomats from several nations met in Geneva to resume peace talks aimed at resolving a long-standing regional conflict. The negotiations, which had been stalled for months, were restarted following a recent de-escalation of hostilities.""",
    
    # 7. Health
    """Researchers have published a landmark study detailing a new gene-editing technique that shows promise in correcting genetic defects responsible for certain inherited diseases. The method uses a modified version of the CRISPR-Cas9 system.""",
    
    # 8. Technology
    """A major tech firm today unveiled its latest flagship smartphone, featuring a revolutionary new camera system with 'periscope zoom' technology. The device, which also boasts a foldable OLED display and 5G connectivity, aims to redefine the premium mobile market.""",
    
    # 9. International Relations
    """A new trade agreement between two major economic blocs was signed this week, aiming to reduce tariffs and streamline regulations across dozens of industries. The pact is the culmination of nearly five years of intense negotiations.""",
    
    # 10. Finance (from previous test)
    """Global stock markets experienced a volatile week as central banks around the world signaled a more aggressive stance on combating inflation. The US Federal Reserve hinted at larger-than-expected interest rate hikes, causing a sell-off in technology stocks."""
]

for i, article in enumerate(articles_to_test, 1):
    print(f"\n\n--- SUMMARIZING ARTICLE {i}/{len(articles_to_test)} ---")
    summarize_hindi(article)



--- SUMMARIZING ARTICLE 1/10 ---

SOURCE ARTICLE:
India secured a decisive victory over Australia in the final match of the T20
series, winning by a margin of 35 runs in Bengaluru. Batting first, India posted
a competitive total of 198 for 4, thanks to a powerful half-century from captain
Suryakumar Yadav.

GENERATED HINDI SUMMARY (v15):
भारत ने बांगलारू में टी20 सीरीज़ के अंतिम मैच में ऑस्ट्रेलिया पर 35 रनों की बढ़त
के साथ निर्णायक जीत हासिल की। पहले बल्लेबाजी करते हुए, भारत ने 4 मैचों में अपनी
प्रतिस्पर्धी कुल संख्या 198 रन बनाए रखी। कप्तान सुकुमार यादव ने इस जीत का श्रेय
एक शक्तिशाली आधा शताब्दी को दिया।



--- SUMMARIZING ARTICLE 2/10 ---

SOURCE ARTICLE:
NASA's Artemis program achieved a major milestone this week as the Orion
spacecraft successfully completed its uncrewed flyby of the Moon and is now on
its return trajectory to Earth. The mission, Artemis I, is a critical test of
the agency's deep space exploration systems.

GENERATED HINDI SUMMARY (v15):
नासा के आर्टेमिस कार्यक

In [13]:
articles_to_test = [
    # 1. Health
    """
    A new study published in the Lancet suggests that moderate, regular exercise,
    such as brisk walking or cycling for 30 minutes a day,
    can significantly reduce the risk of cardiovascular disease.
    The research tracked over 100,000 individuals across 15 countries,
    finding that even non-strenuous activity was highly beneficial
    for long-term heart health, independent of other risk factors.
    """,
    
    # 2. Technology
    """
    Quantum computing researchers at a leading university have announced
    a breakthrough in qubit stability. By developing a new silicon-based
    architecture, they managed to maintain a quantum state for several
    milliseconds, a significant leap from the previous microsecond records.
    This advancement addresses a major hurdle in building scalable,
    fault-tolerant quantum computers.
    """,
    
    # 3. Environment
    """
    Oceanographers are expressing growing concern over the rapid expansion
    of "oxygen minimum zones" (OMZs) in the world's oceans.
    These vast areas, where oxygen levels are too low to support most marine life,
    are being exacerbated by climate change and nutrient runoff from agriculture.
    The expanding dead zones threaten vital fishing industries and marine biodiversity.
    """,
    
    # 4. Space
    """
    The James Webb Space Telescope has captured a stunning new image
    of a distant stellar nursery, revealing intricate details of star formation
    that were previously hidden by dust clouds. The infrared image shows
    dozens of protostars in their earliest stages, offering astronomers
    an unprecedented glimpse into the processes that formed our own solar system.
    """,
    
    # 5. Business
    """
    Semiconductor manufacturing giant TSMC has confirmed plans to build
    a new $40 billion fabrication plant in Arizona. The move is part
    of a broader global trend to diversify the microchip supply chain,
    which has faced significant disruptions over the past few years.
    The new plant is expected to create thousands of high-tech jobs
    but also faces challenges related to water supply and skilled labor.
    """,
    
    # 6. Artificial Intelligence
    """
    AI safety researchers are calling for new regulatory frameworks to govern
    the development of "frontier" AI models. A recently published paper
    argues that as models become more powerful, their potential for misuse
    in cybersecurity and disinformation campaigns grows exponentially.
    The authors propose a system of independent audits and licensing for
    models that exceed a certain capability threshold.
    """,
    
    # 7. World News
    """
    A critical summit on global food security concluded in Rome, with
    leaders pledging to increase funding for sustainable agriculture
    and emergency food aid. The talks were overshadowed by ongoing supply
    chain disruptions and a sharp rise in fertilizer costs, which have
    compounded food shortages in several developing nations.
    """,
    
    # 8. Entertainment
    """
    A 1962 Ferrari 250 GTO has become the most expensive car from the
    automaker ever sold at public auction, fetching a staggering $51.7 million.
    The legendary vehicle, one of only 36 ever built, was the centerpiece
    of a high-profile art and luxury auction in New York.
    The sale underscores the soaring value of rare, classic automobiles
    as tangible assets for high-net-worth collectors.
    """,
    
    # 9. Health & Medicine
    """
    Biotech firm Moderna has announced promising results from its Phase 3
    trial for a combination vaccine targeting both COVID-19 and influenza.
    The single-shot mRNA vaccine reportedly produced a stronger immune
    response against both viruses than separate, co-administered flu
    and COVID shots. If approved, the combination vaccine could simplify
    annual immunization campaigns significantly.
    """,
    
    # 10. Cybersecurity
    """
    A global cybersecurity firm has issued an alert regarding a
    sophisticated new strain of ransomware targeting hospital networks.
    The malware, dubbed "Medusa," not only encrypts critical patient
    data but also exfiltrates it, putting added pressure on healthcare
    providers to pay the ransom. Experts are urging all medical
N-institutions to update their security protocols immediately.
    """
]

for i, article in enumerate(articles_to_test, 1):
    print(f"\n\n--- SUMMARIZING ARTICLE {i}/{len(articles_to_test)} ---")
    summarize_hindi(article)



--- SUMMARIZING ARTICLE 1/10 ---

SOURCE ARTICLE:
     A new study published in the Lancet suggests that moderate, regular
exercise,     such as brisk walking or cycling for 30 minutes a day,     can
significantly reduce the risk of cardiovascular disease.     The research
tracked over 100,000 individuals across 15 countries,     finding that even non-
strenuous activity was highly beneficial     for long-term heart health,
independent of other risk factors.

GENERATED HINDI SUMMARY (v15):
द लांसेट में प्रकाशित एक नए अध्ययन से पता चलता है कि 30 मिनट प्रतिदिन हल्के चलने
या साइकिल चलाने जैसी नियमित गतिविधियां हृदय रोग के जोखिम को काफी हद तक कम कर
सकती हैं। 15 देशों में 100,000 से अधिक व्यक्तियों की निगरानी करते हुए किए गए इस
शोध में पाया गया कि गैर-श्रेष्ठ गतिविधि दीर्घकालिक हृदय स्वास्थ्य के लिए अत्यधिक
लाभदायक थी, जो अन्य जोखिम वाले कारकों से अलग थी।



--- SUMMARIZING ARTICLE 2/10 ---

SOURCE ARTICLE:
     Quantum computing researchers at a leading university have announced     a
br

In [14]:
articles_to_test = [
    # --- Short Article (Sports) ---
    """
    India secured a decisive victory over Australia in the final match of the T20 
    series, winning by a margin of 35 runs in Bengaluru. Batting first, India posted 
    a competitive total of 198 for 4, thanks to a powerful half-century from captain 
    Suryakumar Yadav, who scored 78 off just 45 balls. In response, Australia's 
    chase faltered early as they lost key wickets to India's fast bowlers.
    """,
    
    # --- Medium Article (Technology) ---
    """
    A major tech firm today unveiled its latest flagship smartphone, featuring a 
    revolutionary new camera system with 'periscope zoom' technology. The device, 
    which also boasts a foldable OLED display and 5G connectivity, aims to redefine 
    the premium mobile market. Analysts are optimistic, noting that the innovative 
    camera could be a key differentiator in a crowded field. However, concerns remain 
    about the device's high price point, which exceeds $1,500, potentially 
    limiting its mass-market appeal despite the advanced features.
    """,
    
    # --- Long Article (Science/Space) ---
    """
    NASA's Artemis program achieved a major milestone this week as the Orion 
    spacecraft successfully completed its uncrewed flyby of the Moon and is now on 
    its return trajectory to Earth. The mission, Artemis I, is a critical test of 
    the agency's deep space exploration systems, including the powerful Space Launch 
    System (SLS) rocket and the Orion crew capsule. During its journey, Orion 
    traveled farther from Earth than any human-rated spacecraft has ever gone 
    before, capturing stunning high-resolution images of the lunar surface and 
    Earth from a distance. The spacecraft's heat shield will face its most extreme 
    test during re-entry, when it will endure temperatures of nearly 5,000 
    degrees Fahrenheit while traveling at over 24,000 miles per hour. A successful 
    splashdown in the Pacific Ocean will pave the way for Artemis II, the program's 
    first crewed mission, which will send astronauts on a similar lunar flyby, 
    further cementing humanity's path back to the Moon and, eventually, to Mars.
    """,
    
    # --- Medium Article (Business/Finance) ---
    """
    Global stock markets experienced a volatile week as central banks around the 
    world signaled a more aggressive stance on combating inflation. The US Federal 
    Reserve hinted at larger-than-expected interest rate hikes, causing a sell-off 
    in technology stocks and growth-oriented sectors. Meanwhile, the European 
    Central Bank is facing pressure to act as energy prices continue to soar 
    across the continent, impacting both consumer spending and industrial 
    production. Investors are now closely watching upcoming inflation data and 
    corporate earnings reports for signs of a potential economic slowdown. Experts 
    suggest a period of uncertainty is likely to continue as markets digest the 
    new reality of tighter monetary policy.
    """,

    # --- Short Article (World News) ---
    """
    Diplomats from several nations met in Geneva to resume peace talks aimed at 
    resolving a long-standing regional conflict. The negotiations, which had 
    been stalled for months, were restarted following a recent de-escalation of 
    hostilities. Observers are cautiously optimistic, but acknowledge that 
    significant political hurdles remain. The primary goal of the current round 
    of talks is to establish a lasting ceasefire and facilitate the delivery of 
    humanitarian aid to affected civilian populations.
    """,
    
    # --- Long Article (Health/Science) ---
    """
    Researchers have published a landmark study in the journal 'Nature' detailing 
    a new gene-editing technique that shows promise in correcting genetic defects 
    responsible for certain inherited diseases. The method, which uses a modified 
    version of the CRISPR-Cas9 system, demonstrated a significantly higher precision 
    and lower rate of off-target mutations in lab experiments compared to existing 
    technologies. The study focused on a specific mutation linked to cystic 
    fibrosis, and the results in human cell cultures were highly encouraging. 
    While the research is still in its early stages and human trials are years 
    away, the scientific community is hailing it as a potential breakthrough. 
    The technique's improved safety profile could overcome some of the major hurdles 
    that have slowed the clinical application of gene therapy. However, the 
    scientists involved urge caution, emphasizing that extensive further research 
    is required to validate the findings and ensure the long-term safety and 
    efficacy of this new approach before it can be considered for patient treatment.
    """,

    # --- Medium Article (Environment) ---
    """
    A new international report has found that the rate of deforestation in the 
    Amazon rainforest accelerated by nearly 20% in the last year, reaching its 
    highest level in over a decade. The report, which uses satellite data, 
    attributes the surge to increased illegal logging, agricultural expansion, 
    and mining activities. Environmental groups are calling for urgent government 
    intervention and stronger enforcement of existing protection laws. The Amazon 
    is a critical global ecosystem, playing a vital role in regulating the 
    planet's climate by absorbing vast amounts of carbon dioxide. Scientists 
    warn that continued deforestation could push the rainforest towards a 
    tipping point, where it would transition into a drier, savanna-like state, 
    with devastating consequences for global biodiversity and climate patterns.
    """,

    # --- Short Article (Entertainment) ---
    """
    The highly anticipated sequel to a blockbuster science fiction film has 
    officially begun production, with the studio releasing the first on-set photo. 
    The image features the return of the original cast members alongside several new 
    additions. The director has promised that the sequel will expand the universe 
    in exciting ways while honoring the spirit of the first film. The movie is 
    currently slated for a summer release next year.
    """,

    # --- Medium Article (Automotive/Tech) ---
    """
    A legacy automaker has announced a massive $50 billion investment into its 
    electric vehicle (EV) division, signaling a dramatic acceleration of its 
    transition away from internal combustion engines. The company plans to 
    launch 15 new all-electric models over the next three years, including sedans, 
    SUVs, and a pickup truck. A key part of the strategy involves building 
    several new "gigafactories" for battery production in North America and 
    Europe to secure its supply chain. This move is seen by industry experts 
    as a direct response to the growing dominance of EV-native companies and 
    increasing regulatory pressure to reduce emissions.
    """,

    # --- Long Article (International Relations) ---
    """
    A new trade agreement between two major economic blocs was signed this week, 
    aiming to reduce tariffs and streamline regulations across dozens of industries. 
    The pact, which covers everything from agricultural goods to digital services, 
    is the culmination of nearly five years of intense negotiations. Proponents 
    argue that the deal will boost economic growth, lower consumer prices, and 
    create hundreds of thousands of new jobs by fostering closer integration and 
    simplifying cross-border commerce. However, the agreement has also faced 
    criticism from labor unions and environmental groups, who argue that it 
    lacks sufficient protections for workers' rights and fails to implement 
    strong environmental standards. They warn that the deal could lead to a "race 
    to the bottom" as companies relocate to regions with lower wages and weaker 
    regulations. The signatory governments have defended the pact, stating that 
    it includes robust chapters on labor and the environment and will be subject 
    to regular reviews to ensure compliance and address any emerging issues.
    """,
]

for i, article in enumerate(articles_to_test, 1):
    print(f"\n\n--- SUMMARIZING ARTICLE {i}/{len(articles_to_test)} ---")
    
    summarize_hindi(article)



--- SUMMARIZING ARTICLE 1/10 ---

SOURCE ARTICLE:
     India secured a decisive victory over Australia in the final match of the
T20      series, winning by a margin of 35 runs in Bengaluru. Batting first,
India posted      a competitive total of 198 for 4, thanks to a powerful half-
century from captain      Suryakumar Yadav, who scored 78 off just 45 balls. In
response, Australia's      chase faltered early as they lost key wickets to
India's fast bowlers.

GENERATED HINDI SUMMARY (v15):
भारत ने बांगलारू में टी20 सीरीज़ के अपने अंतिम मैच में ऑस्ट्रेलिया पर 35 रनों की
बढ़त के साथ निर्णायक जीत हासिल की। पहले बल्लेबाजी करते हुए, भारत का कुल स्कोर
198 रन था। कप्तान सुकुमार यादव ने एक शक्तिशाली अर्धशतक बनाया, जिन्होंने केवल 45
गेंदों पर 78 विकेट लिए थे। जवाब में, ऑस्ट्रेलिया ने शुरुआत में महत्वपूर्ण विकेट
गंवा दिए और तेजी से गेंदबाजी करने वाले तेज गेंदबाजों को आउट कर दिया।



--- SUMMARIZING ARTICLE 2/10 ---

SOURCE ARTICLE:
     A major tech firm today unveiled its latest flagship smar

In [11]:
article_to_test = """
Global stock markets experienced a volatile week as central banks 
around the world signaled a more aggressive stance on combating inflation. 
The US Federal Reserve hinted at larger-than-expected interest rate hikes, 
causing a sell-off in technology stocks and growth-oriented sectors. 
Meanwhile, the European Central Bank is facing pressure to act as energy 
prices continue to soar across the continent, impacting both consumer 
spending and industrial production.
"""

summarize_hindi(article_to_test)


SOURCE ARTICLE:
 Global stock markets experienced a volatile week as central banks  around the
world signaled a more aggressive stance on combating inflation.  The US Federal
Reserve hinted at larger-than-expected interest rate hikes,  causing a sell-off
in technology stocks and growth-oriented sectors.  Meanwhile, the European
Central Bank is facing pressure to act as energy  prices continue to soar across
the continent, impacting both consumer  spending and industrial production.

GENERATED HINDI SUMMARY (v15):
वैश्विक शेयर बाजारों ने एक उथल-पुथल का अनुभव किया, जब प्रमुख बैंक विश्व स्तर पर
मुद्रास्फीति को नियंत्रित करने के लिए अधिक आक्रामक रुख अपना रहे थे। अमेरिकी
फेडरल रिजर्व ने अनुमानित ब्याज दरों की बढ़ोतरी की सूचना दी, जिससे तकनीकी शेयरों
और विकास-आधारित क्षेत्रों में खरीदारी हुई। इसके विपरीत, यूरोपीय संघ (ECB) लगातार
बढ़ती ऊर्जा कीमतों के कारण महत्वपूर्ण दबाव का सामना कर रहा है, जो उपभोक्ता खर्च
और औद्योगिक उत्पादन दोनों को प्रभावित कर रही है।



Bulk Testing on Dataset

In [15]:
import torch
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import pandas as pd
import textwrap
import os
import logging
from tqdm import tqdm
from datetime import datetime
import sys 

import config
import data_utils
import logging_utils

# --- Configuration ---
DATA_PATH = r"H:\News_Summarization\Dataset\filtered_articles_CNN.csv" 
NUM_SAMPLES = 10
# ----------------------------------------------

model = None
tokenizer = None

def load_model():
    """Loads the final v15 model and tokenizer."""
    global model, tokenizer
    
    if not os.path.exists(config.FINAL_SAVE_PATH):
        logging.error(f"Model path not found: {config.FINAL_SAVE_PATH}")
        print(f"Error: Model path not found: {config.FINAL_SAVE_PATH}")
        return False

    logging.info(f"Loading final model from: {config.FINAL_SAVE_PATH}...")
    try:
        model = MBartForConditionalGeneration.from_pretrained(config.FINAL_SAVE_PATH).to(config.DEVICE)
        tokenizer = MBart50TokenizerFast.from_pretrained(config.FINAL_SAVE_PATH)
        model.eval()
        logging.info(f"Model v15 loaded successfully on {config.DEVICE}")
        return True
    except Exception as e:
        logging.error(f"Error loading model: {e}", exc_info=True)
        return False

def load_test_data(num_samples=None):
    """Loads, cleans, and subsets the test data from the CSV."""
    logging.info(f"Loading test data from: {DATA_PATH}")
    
    try:
        df = pd.read_csv(DATA_PATH, engine='python', on_bad_lines='skip')
    except FileNotFoundError:
        logging.error(f"Data file not found at: {DATA_PATH}")
        print(f"Error: Data file not found at: {DATA_PATH}")
        return None
        
    df.dropna(subset=['raw_news_article', 'hindi_summary'], inplace=True)
    
    # Clean the text
    df['raw_news_article'] = df['raw_news_article'].apply(data_utils.sanitize_text).apply(data_utils.normalize_text)
    df['hindi_summary'] = df['hindi_summary'].apply(data_utils.sanitize_text).apply(data_utils.normalize_text)
    
    if num_samples and num_samples > 0:
        if num_samples > len(df):
             logging.warning(f"Requested {num_samples} samples, but only {len(df)} are available. Using all articles.")
             num_samples = len(df)
        
        logging.warning(f"Selecting a random subset of {num_samples} articles for testing.")
        df = df.sample(n=num_samples, random_state=42) 
        
    else:
        logging.info(f"Processing all {len(df)} articles found in the file.")
        
    return df

def run_inference_and_log(df):
    """
    Generates summaries for the dataframe, printing results in real-time.
    Returns a list of generated summaries.
    """
    generated_summaries = []
    
    batch_size = 1 
    
    gen_kwargs = {
        "num_beams": 12,
        "length_penalty": 2.0,
        "repetition_penalty": 2.5,
        "no_repeat_ngram_size": 3,
        "do_sample": False,
        "early_stopping": True,
        "min_length": 30,
        "max_length": 250,
    }

    logging.info(f"Starting inference for {len(df)} articles...")
    
    df = df.reset_index(drop=True) 
    
    for index, row in tqdm(df.iterrows(), total=len(df), desc="Summarizing Articles"):
        article = row['raw_news_article']
        
        tokenizer.src_lang = "en_XX"
        inputs = tokenizer(
            article, 
            return_tensors="pt", 
            max_length=1024, 
            truncation=True, 
            padding=True
        ).to(config.DEVICE)

        summary_ids = model.generate(
            inputs.input_ids,
            forced_bos_token_id=tokenizer.lang_code_to_id["hi_IN"],
            **gen_kwargs
        )
        
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        generated_summaries.append(summary)
        
        log_output_console = (
            f"\n\n--- Article {index+1}/{len(df)} ---\n"
            f"\nSOURCE:\n{textwrap.fill(article, width=80)}\n"
            f"\nGENERATED HINDI:\n{textwrap.fill(summary, width=80)}\n"
            f"{'='*80}"
        )
        
        print("\n\n" + log_output_console)
        
        log_output_file = (
            f"\n\n--- Article {index+1}/{len(df)} (Original Index: {row.name}) ---\n"
            f"\nSOURCE:\n{textwrap.fill(article, width=80)}\n"
            f"\nGENERATED HINDI:\n{textwrap.fill(summary, width=80)}\n"
            f"\nREFERENCE HINDI:\n{textwrap.fill(row['hindi_summary'], width=80)}\n"
            f"{'='*80}"
        )
        logging.info(log_output_file)
        
    return generated_summaries

def save_results(df, generated_summaries):
    """Saves the final results to a CSV file."""
    df['generated_hindi_summary'] = generated_summaries
    
    output_df = df[['raw_news_article', 'hindi_summary', 'generated_hindi_summary']]
    
    output_filename = os.path.join(
        config.MODEL_OUTPUT_DIR, 
        f"bulk_test_results_interactive_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.csv"
    )
    
    output_df.to_csv(output_filename, index=False, encoding='utf-8-sig')
    logging.info(f"Successfully saved full results to: {output_filename}")
    print(f"\nSuccessfully saved full results to: {output_filename}")

def main():
    log_path = logging_utils.setup_logging(config.MODEL_OUTPUT_DIR, "bulk_test_interactive_log_v15")
    print(f"--- Starting Bulk Test Script ---")
    print(f"Logging all output to: {log_path}\n")
    logging.info("--- Starting Bulk Test Script ---")
    
    if load_model():
        test_data = load_test_data(NUM_SAMPLES)
        if test_data is not None:
            generated_summaries = run_inference_and_log(test_data)
            save_results(test_data, generated_summaries)
            logging.info("--- Bulk test finished successfully ---")
            print("--- Bulk test finished successfully ---")
        else:
            logging.error("Failed to load test data. Aborting.")

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        logging.warning("--- Bulk Test Interrupted by User ---")
        print("\n\n--- Bulk Test Interrupted by User ---")
        try:
            sys.exit(0)
        except SystemExit:
            os._exit(0)
    except Exception as e:
        logging.error("--- Bulk Test FAILED ---", exc_info=True)
        print(f"\n--- Bulk Test FAILED --- \nError: {e}\nSee log file for details.")
        try:
            sys.exit(1)
        except SystemExit:
            os._exit(1)

loading configuration file mbart-large-50-cnn-summarizer-v15\final_model\config.json
Model config MBartConfig {
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "MBartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "dtype": "float32",
  "early_stopping": null,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL

--- Starting Bulk Test Script ---
Logging all output to: mbart-large-50-cnn-summarizer-v15\bulk_test_interactive_log_v15_2025-11-14_08-51-07.log



loading configuration file mbart-large-50-cnn-summarizer-v15\final_model\generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": [
    2
  ],
  "forced_eos_token_id": 2,
  "max_length": 200,
  "num_beams": 5,
  "pad_token_id": 1
}

Could not locate the custom_generate/generate.py inside mbart-large-50-cnn-summarizer-v15\final_model.
loading file sentencepiece.bpe.model
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading file chat_template.jinja
Summarizing Articles:  10%|█         | 1/10 [00:06<00:55,  6.20s/it]





--- Article 1/10 ---

SOURCE:
Charlie Kirk's suspected killer brought into custody after confessing to father
2 days ago Share Save Christal Hayes and Regan Morris Utah Share Save Utah
governor details how Charlie Kirk murder suspect apprehended A 22-year-old from
Utah has been arrested over the killing of right-wing activist Charlie Kirk, who
was shot dead while on stage at a university event earlier this week. Tyler
Robinson was taken into custody late on Thursday, after a 33-hour manhunt that
ended after his father helped persuade him to surrender to police. His arrest
was first announced by US President Donald Trump, who called for the suspect to
face the death penalty. The killing of Kirk, who was shot while debating with
students on Wednesday, has shocked Americans and laid bare the country's sharp
partisan divisions. Follow live updates Shooting throws Utah students into heart
of US political divide 'I will never let your legacy die' - Charlie Kirk's widow
gives tearful addr

Summarizing Articles:  20%|██        | 2/10 [00:12<00:49,  6.19s/it]

...





--- Article 8/10 ---

SOURCE:
The UK entered WW2 on 3 September, 1939. Eighty years after the war's end, few
people know that a British island's unique language was used as a clandestine
code during Nazi occupation "Bonsouair! Seyiz les beinv'nus!" announced Jo
Thorpe, as she stepped up to the microphone and spread her arms in welcome. "Or
for anyone who's not from Jersey, good evening and welcome! We're thrilled to
see you tonight, and we hope you enjoy the show." The lights dimmed, and her
band, Sonneux, started playing. Accompanied by violin, double bass, guitar and
recorder, Thorpe began to sing a familiar folk melody – only there was something
surprising about her tune. Though it sounded like she was singing in French, in
fact, she was singing in another language entirely: Jèrriais, Jersey's native
language. "In Jersey, we call this a veil'ye," she explained during a mid-show
break, handing me a bowl of bean crock, a traditional, hearty island stew. "It's
an evening of singing

Summarizing Articles:  90%|█████████ | 9/10 [00:47<00:04,  4.43s/it]





--- Article 9/10 ---

SOURCE:
On January 23rd, just three days after John. F. Kennedy delivered his inaugural
speech as the 35th President of the United States, one little known event could
have changed American history in the most catastrophic way imaginable. A
refuelling accident caused a B-52 bomber to break apart above a farm in
Goldsboro, North Carolina, causing two 3.8 megaton nuclear bombs to plummet to
the ground. Bar one small safety switch and a huge amount of luck, millions
could have been killed. Accidents like this are known as 'Broken Arrows' and
they have happened more than many people realise.

GENERATED HINDI:
23 जनवरी को, जॉन एफ. कैनेडी के 35वें राष्ट्रपति के रूप में पदभार संभालने के तीन
दिन बाद, नॉर्थ कैरोलिना के गोल्ड्सबोरो स्थित एक खेत के ऊपर एक विस्फोटक दुर्घटना
हुई। इस घटना ने दो 3.8 मिलियन किलोग्राम वजनी परमाणु बमों को जमीन पर गिरा दिया,
जिससे लाखों लोगों की जान चली गई। ऐसी घटनाओं को 'ब्लॉकन आर्रोज़' कहा जाता है, जो
आमतौर पर अनजाने में होती हैं और जिन्हें कई

Summarizing Articles: 100%|██████████| 10/10 [00:53<00:00,  5.36s/it]





--- Article 10/10 ---

SOURCE:
Malawi - where the petrol queue might overshadow the queue to vote 19 hours ago
Share Save Nomsa Maseko BBC Africa, Lilongwe and Mangochi Share Save BBC The
queue to get fuel rather than the queue to vote is what is on the mind of many
Malawians as Tuesday's general election approaches. Prolonged petrol shortages,
along with regular power cuts, the rising cost of living, hunger, poverty,
inequality and youth unemployment, add to the tangible frustration here. The
presidential, parliamentary and local council candidates are competing for votes
against a background of cynicism about what might actually change. In a sign
that money is tight, electioneering has been somewhat muted compared to the
past. This is despite the presidential race being seen as a rematch between the
incumbent, Lazarus Chakwera, and the man he beat in 2020, then-President Peter
Mutharika. There are 15 other candidates. But the usual colourful campaign
carnival is missing. The free